In [115]:
import serial
import threading
import time
from PyQt5.QtGui import QImage, qGray
from PyQt5.QtCore import QSize, Qt




In [116]:
remain_cmd = None
cmd_todecode = None
check_online = False
askToStop = False
uploadInProcess = False
carvingPercentProgress = 100
numericUpDown_times = 1
fan_RPM = 0
fan_Precent = 0



In [117]:
def decode(D1, D2, D3):
        # print(D1, D2, D3)
        if D1 == 0:
            print("ONLINE")
            online = True
            return
        # elif D1 == 1:
        #    break
        elif D1 == 2:
            print("CONNECTION OK !")

In [118]:
def dataReceived(s):

    remain_cmd = None # sacar, deberia ser parametro local


    nbToRead = s.inWaiting()
    if nbToRead == 0:
        return False
    #
    data = bytearray(s.read(nbToRead))
    #
    if remain_cmd is not None:
        cmd_todecode = bytearray(remain_cmd)
        cmd_todecode.extend(data)
    else:
        cmd_todecode = bytearray(data)
    remain_cmd = None

    if (len(cmd_todecode) % 4 != 0):
        remain_cmd = bytearray(len(cmd_todecode) % 4)
        lr = len(remain_cmd)
        lt = len(cmd_todecode)
        for i in range(len(remain_cmd)):
            remain_cmd[i] = cmd_todecode[lt - lr + i]
        array2 = bytearray(cmd_todecode)
        cmd_todecode = bytearray(len(array2) - lr)
        for j in range(len(array2) - lr):
            cmd_todecode[j] = array2[j]
    cmd = cmd_todecode
    for k in range(len(cmd)):
        if (cmd[k] == 255) and (k+3) < len(cmd):
            decode(cmd[k + 1], cmd[k + 2], cmd[k + 3])
    return True


In [119]:
def wait_data(s):
    finish = False
    while not finish:
        finish = dataReceived(s)
        time.sleep(0.02)

In [120]:
def send_CMD(s, D1, D2, D3):
    buffer = bytes([255, D1, D2, D3])
    print("send", buffer)
    s.write(buffer)
    s.flush()


 

In [121]:
def send_CMD_array(s, arr):
    print("send", arr)
    buffer = bytes([255]+arr)
    print("send", buffer)
    s.write(buffer)
    s.flush()

In [122]:
serial = serial.Serial("/dev/cu.usbserial-1140", 57600)

if serial.isOpen():
        # #FF#09#5A#A5
        send_CMD(serial, 9, 90, 165)                                    # se conecta
        wait_data(serial)                                               # responde CONNECTION OK !
        # #FF#AA#08#01#01#5A#A5#55
        send_CMD_array(serial, [170, 8, 1, 1, 90, 165, 85])             # este parece que hace home
        serial.write(b"\xFB") # move center
        [serial.write(b"\xF5\x01") for _ in range(0, 15)]
else:
        print("Error : no serial connection")



send b'\xff\tZ\xa5'
send [170, 8, 1, 1, 90, 165, 85]
send b'\xff\xaa\x08\x01\x01Z\xa5U'


In [124]:

[serial.write(b"\xF5\x02") for _ in range(0, 15)]



[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [101]:
serial.write(b"\xF9") # reset

1

In [126]:
serial.write(b"\xF6")

1

In [1]:
import serial
usb_port = "/dev/cu.usbserial-1140"
baud_rate = 57600

#serial = serial.Serial("/dev/cu.usbserial-1140", 57600)

ser = serial.Serial(
            port=usb_port,
            baudrate=baud_rate,
            parity=serial.PARITY_ODD,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS,
)

ser.write(b"\xF6") # 92 120 70 54
res = ser.read(2)
if res == b"e\xfb":
    print("nejemojo is go")
else:
    print(f"bad mojo: {res}")



bad mojo: b'\xbc\xbc'


In [2]:
print(b'\xF6')

b'\xf6'
